In [1]:
import numpy as np
from datetime import datetime
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV
from sklearn.metrics import f1_score, precision_score, precision_recall_curve, recall_score, accuracy_score, roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer

In [2]:
#Import the data 
# read all tables 
customer_final = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/customers_final.csv')
engagement_final = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/engagements_final.csv')
marketing_final = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/marketing_final.csv')
transactions_final = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/transactions_final.csv')

In [3]:
# deal with gender null 
# deal with age null
customer_final.dropna(subset=['gender', 'age'], inplace=True)

In [4]:
customer_final['join_date'] = pd.to_datetime(customer_final['join_date'])
customer_final['last_purchase_date'] = pd.to_datetime(customer_final['last_purchase_date'])
transactions_final['transaction_date'] = pd.to_datetime(transactions_final['transaction_date'])
marketing_final['campaign_date'] = pd.to_datetime(marketing_final['campaign_date'])

feature engineering 

In [8]:
#creating CLV dataset 
clv_final = transactions_final.groupby('customer_id')['transaction_amount'].sum().reset_index()
clv_final.rename(columns={'transaction_amount': 'clv'}, inplace=True)

In [16]:
transactions_agg = transactions_final.groupby('customer_id').aggregate({'transaction_id':'count','transaction_amount':'sum'})

In [25]:
transactions_agg 

,transaction_id,transaction_amount
customer_id,,
1,6,3509.48
2,9,6081.32
3,6,1454.87
4,20,7874.68
5,24,15524.55
...,...,...
9996,10,5498.20
9997,12,5848.30
9998,3,3503.13


In [17]:
engagement_final.set_index('customer_id', inplace=True)
marketing_final.set_index('customer_id', inplace=True)

In [22]:
customer_final.set_index('customer_id', inplace=True)

KeyError: "None of ['customer_id'] are in the columns"

In [26]:
joint_data = customer_final.join(engagement_final).join(transactions_agg).join(marketing_final)

In [27]:
joint_data

,join_date,last_purchase_date,age,gender,location,number_of_site_visits,number_of_emails_opened,number_of_clicks,transaction_id,transaction_amount,campaign_id,response,promotion_type,campaign_date
customer_id,,,,,,,,,,,,,,
1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,10,15,1,6,3509.48,1,No,Buy One Get One,2024-02-29
1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,10,15,1,6,3509.48,2,No,Discount,2024-01-24
1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,10,15,1,6,3509.48,3,No,Free Shipping,2024-03-05
1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,10,15,1,6,3509.48,4,Yes,Buy One Get One,2024-01-10
4,2022-01-01,2022-09-01,29.0,Male,Grossstad,110,30,17,20,7874.68,11,No,Free Shipping,2022-06-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9998,2023-09-17,2024-01-30,39.0,Male,New John,53,17,5,3,3503.13,25196,Yes,Discount,2023-09-20
9999,2022-05-10,2022-07-15,31.0,Female,Andrewland,23,5,4,12,6721.86,25197,No,Buy One Get One,2022-06-02
9999,2022-05-10,2022-07-15,31.0,Female,Andrewland,23,5,4,12,6721.86,25198,No,Discount,2022-05-14
